In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import xgboost as xgb
from sklearn.metrics import f1_score

In [2]:
train = pd.read_csv('train_groups.csv')

In [3]:
doc_to_title = {}
with open('titles_norm_ru.csv', encoding='utf-8', newline='') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [4]:
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [5]:
Y_TRAIN = []
X_TRAIN = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        Y_TRAIN.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_TRAIN.append(sorted(all_dist, reverse=True)[0:20])
X_TRAIN = np.array(X_TRAIN)
Y_TRAIN = np.array(Y_TRAIN)
groups_train = np.array(groups_train)
print (X_TRAIN.shape, Y_TRAIN.shape, groups_train.shape)

(11690, 20) (11690,) (11690,)


In [7]:
k=25
group_kfold = GroupKFold(n_splits=k)
res=[]
for train_index, test_index in group_kfold.split(X_TRAIN, Y_TRAIN, groups_train):
    X_train, X_test = X_TRAIN[train_index], X_TRAIN[test_index]
    y_train, y_test = Y_TRAIN[train_index], Y_TRAIN[test_index]
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test)
    param = {'max_depth':2, 'eta':0.02, 'silent':1, 'alpha':0.5, 'lambda':1, 'objective':'binary:logistic' }
    num_round = 175
    bst = xgb.train(param, dtrain, num_round)
    pred = bst.predict(dtest)
    for i in range(len(pred)):
        if pred[i] > 0.36:
            pred[i] = 1
        else:
            pred[i] = 0
    res.append(f1_score(y_test, pred))
qual = np.mean(res)
qual

0.6271077746057202